In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import loadOSCdata
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import psycopg2
import osgeo.ogr
import shapely
import shapely.wkt
%matplotlib inline

In [2]:
data = loadOSCdata.downloadData(OSCid=401441, X = True, Y = True, Z = True, output = 'csv', outputFile = 'data.csv')
data.head()

,timestamp,point_id,v_value,geometry,image_url,image_lab,trip_id
359,2017-05-30 10:27:18.191175,0.0,10.769757,POINT (-73.974177 40.6807485),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
464,2017-05-30 10:27:22.122909,1.0,10.747140,POINT (-73.97433599999999 40.68077),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
548,2017-05-30 10:27:25.199758,2.0,4.727365,POINT (-73.97447149999999 40.6807915),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
630,2017-05-30 10:27:28.167336,3.0,2.349433,POINT (-73.97454999999999 40.6807985),http://storage4.openstreetcam.org/files/photo/...,bad,401441
741,2017-05-30 10:27:32.227059,4.0,6.292053,POINT (-73.974553 40.680786),http://storage4.openstreetcam.org/files/photo/...,good,401441


In [3]:
connect_str = "dbname='squidbike' user='squidbike' \
host='rds-postgresql-10mintutorial.cbz1xmmdmpva.us-east-2.rds.amazonaws.com' \
port='5432' password='squidbikesql'"
conn = psycopg2.connect(connect_str)

In [4]:
cursor = conn.cursor()
cursor.execute("SELECT * from accelerometer2")
rows = cursor.fetchall()

In [ ]:
'''
link to aws PostGIS installation:

http://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/Appendix.PostgreSQL.CommonDBATasks.html#Appendix.PostgreSQL.CommonDBATasks.PostGIS

'''

In [6]:
cursor.execute("create extension postgis")

In [7]:
#cursor.execute("DROP TABLE IF EXISTS bike_trip_testing")
cursor.execute("CREATE TABLE bike_trip_testingFelipe \
               (id SERIAL PRIMARY KEY,\
               timestamp Text,\
               point_id numeric(10,5),\
               v_value numeric(10,5),\
               geometry Geometry,\
               image_url Text,\
               image_lab Text,\
               trip_id Text\
               )")


In [8]:
cursor.execute("CREATE INDEX postgisIndex ON bike_trip_testingFelipe USING GIST(Geometry)")

In [10]:
data.head()

,timestamp,point_id,v_value,geometry,image_url,image_lab,trip_id
359,2017-05-30 10:27:18.191175,0.0,10.769757,POINT (-73.974177 40.6807485),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
464,2017-05-30 10:27:22.122909,1.0,10.747140,POINT (-73.97433599999999 40.68077),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
548,2017-05-30 10:27:25.199758,2.0,4.727365,POINT (-73.97447149999999 40.6807915),http://storage4.openstreetcam.org/files/photo/...,acceptable,401441
630,2017-05-30 10:27:28.167336,3.0,2.349433,POINT (-73.97454999999999 40.6807985),http://storage4.openstreetcam.org/files/photo/...,bad,401441
741,2017-05-30 10:27:32.227059,4.0,6.292053,POINT (-73.974553 40.680786),http://storage4.openstreetcam.org/files/photo/...,good,401441


In [13]:
index=0
timestamp = data.timestamp.iloc[index]

In [15]:
#Saving records in the table
for index in range(len(data)):
    timestamp = data.timestamp.iloc[index]
    
    point_id = data.point_id.iloc[index]
    v_value = data.v_value.iloc[index]
    geometry = data.geometry.iloc[index]
    image_url = data.image_url.iloc[index]
    image_lab = data.image_lab.iloc[index]
    trip_id = data.trip_id.iloc[index]
    cursor.execute("INSERT INTO bike_trip_testingFelipe (timestamp, point_id, v_value, geometry, image_url, image_lab,\
    trip_id) \
    VALUES ('%s', '%d', '%d', ST_GeomFromText('%s', 4326), '%s',\
    '%s', '%s')"\
    %(timestamp, point_id, v_value, geometry, image_url, \
    image_lab, trip_id))
#     if index == 1000:
#         break
conn.commit()

In [18]:
#v_value should be float and for some reason it looses the data after the decimal point

In [17]:
cursor.execute("SELECT * from bike_trip_testingFelipe")
rows = cursor.fetchall()
rows

[(1,
  '2017-05-30 10:27:18.191175',
  Decimal('0.00000'),
  Decimal('10.00000'),
  '0101000020E6100000FCE07CEA587E52C0892650C422574440',
  'http://storage4.openstreetcam.org/files/photo/2017/5/30/proc/401441_3b530_592d8a896b182.jpg',
  'acceptable',
  '401441'),
 (2,
  '2017-05-30 10:27:22.122909',
  Decimal('1.00000'),
  Decimal('10.00000'),
  '0101000020E610000030D461855B7E52C0890CAB7823574440',
  'http://storage4.openstreetcam.org/files/photo/2017/5/30/proc/401441_a9b9c_592d8a89f1f27.jpg',
  'acceptable',
  '401441'),
 (3,
  '2017-05-30 10:27:25.199758',
  Decimal('2.00000'),
  Decimal('4.00000'),
  '0101000020E610000094D8B5BD5D7E52C088F2052D24574440',
  'http://storage4.openstreetcam.org/files/photo/2017/5/30/proc/401441_ba53c_592d8a8a891cd.jpg',
  'acceptable',
  '401441'),
 (4,
  '2017-05-30 10:27:28.167336',
  Decimal('3.00000'),
  Decimal('2.00000'),
  '0101000020E61000004694F6065F7E52C03B55BE6724574440',
  'http://storage4.openstreetcam.org/files/photo/2017/5/30/proc/401441_9